In [ ]:
from datetime import datetime
import os
import json
import requests
import hmac
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time




: 

In [ ]:
file = "df_2022_hr_capped.csv"

df_2022_hr = pd.read_csv(file)
df_2022_hr

: 

In [ ]:
# create a graph object
fig = go.Figure()

#set up layout
fig.update_layout(
    title={
        'text' : "BTCUSD",
        "x" : 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Date",
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

#add candletstick
fig.add_trace(
    go.Candlestick(
        x=df_2022_hr['date'],
        open=df_2022_hr['open'],
        high=df_2022_hr['high'],
        low=df_2022_hr['low'],
        close=df_2022_hr['close'],
        name="BTCUSD"
    )
)


fig.add_trace(
    go.Scatter(x=df_2022_hr['date'], y=df_2022_hr['SMA'], name="SMA(48)")
)


fig.show()

: 

In [ ]:
#sample visual for liquidation levels: need EMA does not work with SMA
Fig2 = go.Figure()

#set up layout
Fig2.update_layout(
    title={
        'text' : "BTCUSD",
        "x" : 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Date",
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

#add candletstick
Fig2.add_trace(
    go.Candlestick(
        x=df_2022_hr['date'],
        open=df_2022_hr['open'],
        high=df_2022_hr['high'],
        low=df_2022_hr['low'],
        close=df_2022_hr['close'],
        name="BTCUSD"
    )
)


Fig2.add_trace(
    go.Scatter(x=df_2022_hr['date'], y=df_2022_hr['25x_liq_long'], name="25x Rolling Liqs(long)"),
)

Fig2.add_trace(
  go.Scatter(x=df_2022_hr['date'], y=df_2022_hr['25x_liq_short'], name="25x Rolling Liqs(short)")
)

Fig2.show()


: 

In [ ]:
%matplotlib notebook
import pandas_datareader as pdr
import datetime as dt
import seaborn as sns
%matplotlib inline

df['hour']=df.date.dt.hour
df['hour']=df['hour'].astype(float)
df.dtypes

data = df.set_index('date')
data

high_low = data['high']-data['low']
high_cp = np.abs(data['high']-data['close'].shift())
low_cp = np.abs(data['low']-data['close'].shift())

df_volatility = pd.concat([high_low, high_cp, low_cp], axis =1)

#calc true range
true_range = np.max(df_volatility, axis =1)

#calc avg true range
average_true_range = true_range.rolling(14).mean()

data['average_true_range']=average_true_range
data

#plot chart
fig, ax = plt.subplots()
average_true_range.plot(ax=ax)
ax2 = data['close'].plot(ax=ax, secondary_y=True, alpha=.3)
ax.set_ylabel('ATR')
ax2.set_ylabel('Price')

plt.axvspan(datetime(2022, 7, 1, 0), datetime(2022,7, 1, 7), color ='lightgray', alpha=0.5)
plt.axvspan(datetime(2022,7,1,8), datetime(2022,7,1,16), color ='lightblue', alpha=0.5)
plt.axvspan(datetime(2022,7,1,17), datetime(2022,7,2,0), color ='lightcoral', alpha=0.5)
plt.tight_layout()

average_true_range_scatter = data['average_true_range']
hour_scatter=data['hour']

x_values = hour_scatter
y_values = average_true_range_scatter
(slope, intercept, rvalue, pvalue, stderr)=linregress(x_values,y_values)
regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x +" + str(round(intercept, 2))

plt.scatter(x_values,y_values)
plt.plot(x_values, regress_values, "r-")

plt.title("Is there a correlation between volatility and time of day?")
plt.xlabel("time of day")
plt.ylabel("ATR")
plt.show()

volume_scatter = data['volume']
hour_scatter=data['hour']

x_axis = hour_scatter
y_axis = volume_scatter

plt.bar(x_axis, y_axis, color ="b", align ='center', width = .5)

#plt.rcParams["figure.figsize"] = [50, 50]
plt.title("What time of day does the majority of trading occur?")
plt.xlabel("time of day")
plt.ylabel("Volume (100's MM)")
plt.tight_layout()

plt.show()


: 